In [19]:
import json
import os
import pandas as pd
import sys
import tensorflow as tf

from lib.dcrnn_utils import load_graph_data
from model.dcrnn_supervisor import DCRNNSupervisor

flags = tf.app.flags
FLAGS = flags.FLAGS

# flags.DEFINE_bool('use_cpu_only', False, 'Whether to run tensorflow on cpu.')


def run_dcrnn(traffic_reading_df):
    run_id = 'dcrnn_DR_2_h_4_64-64_lr_0.01_bs_16_d_0.00_sl_4_MAE_0419163048'

    log_dir = os.path.join('data/model', run_id)

    config_filename = 'config_00.json'
    graph_pkl_filename = 'data/sensor_graph/adj_mat.pkl'
    with open(os.path.join(log_dir, config_filename)) as f:
        config = json.load(f)
    tf_config = tf.ConfigProto()
    if FLAGS.use_cpu_only:
        tf_config = tf.ConfigProto(device_count={'GPU': 0})
    tf_config.gpu_options.allow_growth = True
    _, _, adj_mx = load_graph_data(graph_pkl_filename)
    with tf.Session(config=tf_config) as sess:
        supervisor = DCRNNSupervisor(traffic_reading_df, config=config, adj_mx=adj_mx)
        supervisor.restore(sess, config=config)
#         df_preds = supervisor.test_and_write_result(sess, config['global_step'])
#         for horizon_i in df_preds:
#             df_pred = df_preds[horizon_i]
#             filename = os.path.join('data/results/', 'dcrnn_prediction_%d.h5' % (horizon_i + 1))
#             df_pred.to_hdf(filename, 'results')
#         print('Predictions saved as data/results/dcrnn_seq2seq_prediction_[1-12].h5...')


# def run_fc_lstm(traffic_reading_df):
#     run_id = 'fclstm_h_12_256-256_lr_0.0001_bs_64_d_0.00_sl_12_MAE_1026175218'
#     log_dir = os.path.join('data/model', run_id)
#
#     config_filename = 'config_59.json'
#     # graph_pkl_filename = 'data/sensor_graph/sensor_graph_exp.pkl'
#     with open(os.path.join(log_dir, config_filename)) as f:
#         config = json.load(f)
#     tf_config = tf.ConfigProto()
#     if FLAGS.use_cpu_only:
#         tf_config = tf.ConfigProto(device_count={'GPU': 0})
#     tf_config.gpu_options.allow_growth = True
#     # Set small entries to zero for sparseness.
#     with tf.Session(config=tf_config) as sess:
#         supervisor = LSTMSeq2SeqSupervisor(traffic_reading_df, config=config)
#         supervisor.restore(sess, config=config)
#         df_preds = supervisor.test_and_write_result(sess, config['global_step'])
#         for horizon_i in df_preds:
#             df_pred = df_preds[horizon_i]
#             # filename = os.path.join('data/results/', 'gcrnn_seq2seq_prediction_%d.h5' % (horizon_i + 1))
#             filename = os.path.join('data/results/', 'fc_lstm_prediction_%d.h5' % (horizon_i + 1))
#             df_pred.to_hdf(filename, 'results')
#         print('Predictions saved as data/results/fc_lstm_prediction_[1-12].h5...')


if __name__ == '__main__':
    sys.path.append(os.getcwd())
    traffic_df_filename = 'data/weather_ts.csv'
    traffic_reading_df = pd.read_csv(traffic_df_filename)
    traffic_reading_df = traffic_reading_df.set_index('time_stamp')
    traffic_reading_df = traffic_reading_df[6486:]
    run_dcrnn(traffic_reading_df)
    # run_fc_lstm(traffic_reading_df)


2018-04-19 16:42:16,476 - INFO - Log directory: data/model/dcrnn_DR_2_h_4_64-64_lr_0.01_bs_16_d_0.00_sl_4_MAE_0419163048/
2018-04-19 16:42:16,476 - INFO - Log directory: data/model/dcrnn_DR_2_h_4_64-64_lr_0.01_bs_16_d_0.00_sl_4_MAE_0419163048/
2018-04-19 16:42:16,480 - INFO - {u'verbose': 0, u'num_rnn_layers': 2, u'min_learning_rate': 2e-06, u'epochs': 100, u'patience': 50, u'test_ratio': 0.2, u'add_time_in_day': False, u'cl_decay_steps': 2000, u'write_db': False, u'epoch': 0, u'max_diffusion_step': 2, u'lr_decay_epoch': 20, u'learning_rate': 0.01, u'log_dir': u'data/model/dcrnn_DR_2_h_4_64-64_lr_0.01_bs_16_d_0.00_sl_4_MAE_0419163048/', u'validation_ratio': 0.1, u'data_type': u'ALL', u'dropout': 0.0, u'filter_type': u'dual_random_walk', u'batch_size': 8, u'graph_pkl_filename': u'data/sensor_graph/adj_mat.pkl', u'max_grad_norm': 5.0, u'model_filename': u'data/model/dcrnn_DR_2_h_4_64-64_lr_0.01_bs_16_d_0.00_sl_4_MAE_0419163048/models-6.9230-276', u'global_step': 276, u'use_cpu_only': Fal

ValueError: Variable DCRNN/learning_rate already exists, disallowed. Did you mean to set reuse=True in VarScope? Originally defined at:

  File "model/tf_model.py", line 38, in __init__
    trainable=False)
  File "model/dcrnn_model.py", line 16, in __init__
    super(DCRNNModel, self).__init__(config, scaler=scaler)
  File "model/dcrnn_supervisor.py", line 83, in _build_train_val_test_models
    adj_mx=self._adj_mx)


In [9]:
traffic_reading_df

,time_stamp,aqi_W San Gabriel Vly,aqi_E San Fernando Vly,aqi_SW Coastal LA,aqi_San Gabriel Mts,aqi_SW San Bernardino,aqi_Southeast LA CO,aqi_South Coastal LA,aqi_Central LA CO,aqi_NW Coastal LA,aqi_Santa Clarita Vly,aqi_W San Fernando Vly,aqi_E San Gabriel V-2
6486,2017-08-01 01:00:00,53.0,75.0,46.0,66.0,59.0,43.0,46.0,75.0,46.0,57.0,75.0,66.0
6487,2017-08-01 02:00:00,50.0,76.0,49.0,60.0,61.0,38.0,49.0,76.0,49.0,53.0,76.0,60.0
6488,2017-08-01 03:00:00,57.0,74.0,48.0,49.0,70.0,38.0,48.0,74.0,48.0,48.0,74.0,49.0
6489,2017-08-01 04:00:00,60.0,70.0,50.0,54.0,60.0,40.0,50.0,70.0,50.0,54.0,70.0,36.0
6490,2017-08-01 05:00:00,59.0,73.0,53.0,44.0,64.0,52.0,53.0,73.0,53.0,44.0,73.0,40.0
6491,2017-08-01 06:00:00,59.0,72.0,56.0,57.0,64.0,45.0,56.0,72.0,56.0,57.0,72.0,42.0
6492,2017-08-01 07:00:00,61.0,72.0,50.0,58.0,49.0,43.0,50.0,72.0,50.0,58.0,72.0,52.0
6493,2017-08-01 08:00:00,58.0,74.0,57.0,58.0,62.0,41.0,57.0,74.0,57.0,58.0,74.0,54.0
6494,2017-08-01 09:00:00,64.0,76.0,50.0,59.0,62.0,43.0,50.0,76.0,50.0,57.0,76.0,59.0
6495,2017-08-01 10:00:00,70.0,79.0,48.0,63.0,60.0,52.0,48.0,79.0,48.0,63.0,79.0,53.0
